## [1] kaggle 사용을 위한 setting

In [0]:
# KAGGLE 증명서 등록하기

!ls -lha kaggle.json -      # json이 폴더 내에 있는지 확인

!pip uninstall -y kaggle      # 캐글 관련 패키지를 다운받기
!pip install --upgrade pip 
!pip install kaggle==1.5.6    # 캐글 버전 맞추기

ls: cannot access '-': No such file or directory
-rw-r--r-- 1 root root 64 May  3 03:29 kaggle.json
Found existing installation: kaggle 1.5.6
Uninstalling kaggle-1.5.6:
  Successfully uninstalled kaggle-1.5.6
Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (20.1)
Processing /root/.cache/pip/wheels/01/3e/ff/77407ebac3ef71a79b9166a8382aecf88415a0bcbe3c095a01/kaggle-1.5.6-py3-none-any.whl


In [0]:
# kaggle.json 을 사용하는 서버.환경에 직접 넣어주기

!mkdir -p ~/.kaggle                   #캐글 폴더 생성
!cp kaggle.json ~/.kaggle/            # 폴더에 kaggle.json을 복붙하기
! chmod 600 ~/.kaggle/kaggle.json     # 복붙한 것의 권한을 주기
!kaggle -v

Kaggle API 1.5.6


In [0]:
# 리눅스 kaggle 주소 복붙하면 challenge의 데이터셋을 다운받을 수 있다.
# 앞에 ! 느낌표를 붙여야 함.

!kaggle competitions download -c classification-animal-classification

!unzip classification-animal-classification.zip    #zip 파일 풀기
                                                    

#이걸로 다운받기위해서는 위에서 challenge 참가를 증명해야 한다.
# 이는 증명서를 다운받아서 서버에 등록해야하기 때문에

classification-animal-classification.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  classification-animal-classification.zip
replace test_des.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

## [2] 데이터 로더

In [0]:
import numpy as np
import torch
import torch.optim as optim
import pandas as pd

torch.manual_seed(1)

xy=pd.read_csv("train.csv", header=None)
                               #헤더 none은 데이터에 따라 다르다.

x_data=xy.loc[:,0:15]
y_data=xy[16]       # pandas_xy.loc[:,16]

x_data=np.array(x_data)
y_data=np.array(y_data)

x_train = torch.FloatTensor(x_data)
y_train = torch.LongTensor(y_data)

## [3] 모델 학습

In [0]:
import torch.nn.functional as F  # for softmax

# 모델 초기화
# feature 16개, 7개의 클래스

nb_class = 7
nb_data = len(y_train)

W = torch.zeros((16, nb_class), requires_grad=True)  
b = torch.zeros(1, requires_grad=True)

# optimizer 설정
optimizer = optim.SGD([W, b], lr=1)   #lr=0.01

nb_epochs = 10000                     #epochs = 1000
for epoch in range(nb_epochs + 1):

    # Cost 계산 (1)
           # softmax함수를 호출하거나, cross-entropy를 그대로 써도 됌.
    hypothesis = F.softmax(x_train.matmul(W) + b, dim=1) # or .mm or @
    
    # cost 표현번 1번 예시
      # one-hot encoding.
    y_one_hot = torch.zeros(nb_data, nb_class)
    y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
    cost = (y_one_hot * -torch.log(F.softmax(hypothesis, dim=1))).sum(dim=1).mean()


    # cost로 H(x) 개선 (w 업데이트)
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 1000 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

  #scatter() 할 때, unsqueeze()는 하지 않음. 이미 클래스의 값이 각 리스트

# lr=10,  에폭 10000 Cost: 1.177567에서 감소하지 않음
# lr=10,  에폭 1000 Cost: 1.177567에서 감소하지 않음

# --> lr=1, 에폭 10000 Cost: 1.167862

Epoch    0/10000 Cost: 1.945910
Epoch 1000/10000 Cost: 1.275906
Epoch 2000/10000 Cost: 1.273207
Epoch 3000/10000 Cost: 1.272380
Epoch 4000/10000 Cost: 1.271953
Epoch 5000/10000 Cost: 1.271685
Epoch 6000/10000 Cost: 1.271491
Epoch 7000/10000 Cost: 1.271136
Epoch 8000/10000 Cost: 1.169481
Epoch 9000/10000 Cost: 1.167594
Epoch 10000/10000 Cost: 1.166951


## [4] 정답 예측

In [0]:
# 학습된 W,b를 통한 클래스 예측

hypothesis = F.softmax(x_train.matmul(W) + b, dim=1) # or .mm or @
predict = torch.argmax(hypothesis, dim=1) 
          # torch.argmax(): hypothesis(=wx)에서 가장 큰 값의 인덱스를 return


#print(hypothesis) # 범주형데이터로 바뀌기 전. wx를 구한 상태.
print(predict)    # 예측한 클래스 
print(y_train)    # 실제 데이터.

# 정확도 계산 
correct_prediction = predict.float() == y_train
print(correct_prediction)
accuracy = correct_prediction.sum().item() / len(correct_prediction)
print('The model has an accuracy of {:2.2f}% for the training set.'.format(accuracy * 100))

# 정확도는 100%

tensor([0, 0, 3, 0, 0, 0, 0, 3, 3, 0, 0, 1, 3, 6, 6, 6, 1, 0, 3, 0, 1, 1, 0, 1,
        5, 4, 4, 0, 0, 0, 5, 0, 0, 1, 3, 0, 0, 1, 3, 5, 5, 1, 5, 1, 0, 0, 6, 0,
        0, 0, 0, 5, 4, 6, 0, 0, 1, 1, 1, 1, 3, 3, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        6, 3, 0, 0, 2, 6, 1, 1, 2], grad_fn=<NotImplemented>)
tensor([0, 0, 3, 0, 0, 0, 0, 3, 3, 0, 0, 1, 3, 6, 6, 6, 1, 0, 3, 0, 1, 1, 0, 1,
        5, 4, 4, 0, 0, 0, 5, 0, 0, 1, 3, 0, 0, 1, 3, 5, 5, 1, 5, 1, 0, 0, 6, 0,
        0, 0, 0, 5, 4, 6, 0, 0, 1, 1, 1, 1, 3, 3, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        6, 3, 0, 0, 2, 6, 1, 1, 2])
tensor([True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, Tr

## [5] 데이터에 대한 TEST


- test_des.csv 를 통해서 학습한 파라미터를 활용해서 test의 라벨은 무엇인지 만들고 제출하기 


In [0]:
test=pd.read_csv('test_des.csv', header=None)
#import pdb;pdb.set_trace()

x_data=test.loc[:, 0:15]     # test_des.csv 에는 라벨이 없기때문에 라벨에 관한것을 하면 오류가 뜰 것임

#판다스x_data ->넘파이 -> tesnsor
x_data=np.array(x_data)
x_test=torch.FloatTensor(x_data)  

hypothesis=F.softmax(x_test.matmul(W) +b, dim=1)
predict=torch.argmax(hypothesis, dim=1)

In [0]:
predict          #tensor형태의 20개 predict가 출력

tensor([1, 3, 1, 0, 6, 3, 1, 5, 4, 1, 4, 3, 0, 0, 1, 0, 5, 0, 1, 1],
       grad_fn=<NotImplemented>)

- 라벨 예측한 predict를 가지고 csv를 만들고 제출하기
- predict를 csv의 형태로 맞추기

In [0]:
# tensor인 predict를 numpy로 바꾸고, reshape()
# id도 0~20까지 만들고 reshape()

# id와 category 헤더를 가지는 데이터프레임
# result를 데이터프레임에 저장(컬럼은 id와 category)

# 데이터프레임을 csv에 저장.   
    #index=False로 하지 않으면 index값까지 들어가게 되고
    #header를 True로 하지 않으면 라벨이 들어가지 않게되어 제출 시 오류가 뜰 수도 있다.

In [0]:
#predict = predict.numpy().reshape(-1,1)
#import pdb;pdb.set_trace()

predict= predict.detach().numpy().reshape(-1,1)

id=np.array([i for i in range(20)]).reshape(-1,1)

result=np.hstack([id,predict])
df=pd.DataFrame(result, columns=["id","Category"])
df.to_csv('result.csv', index=False, header=True)



In [0]:
result

array([[ 0,  1],
       [ 1,  3],
       [ 2,  1],
       [ 3,  0],
       [ 4,  6],
       [ 5,  3],
       [ 6,  1],
       [ 7,  5],
       [ 8,  4],
       [ 9,  1],
       [10,  4],
       [11,  3],
       [12,  0],
       [13,  0],
       [14,  1],
       [15,  0],
       [16,  5],
       [17,  0],
       [18,  1],
       [19,  1]])

In [0]:
 # 캐글 competition에 코드 제출하기
 # kaggle competitions submit -c 제출challenge이름 -f 파일이름.csv -m "제출 메모"
 !kaggle competitions submit -c classification-animal-classification -f result.csv -m "16010064 이주희"

100% 102/102 [00:00<00:00, 318B/s]
Successfully submitted to Classification : Animal Classification